In [39]:
import numpy as np
import pandas as pd
import matplotlib as plt
from IPython.display import clear_output

In [40]:
import tensorflow as tf

In [56]:
csv_column_names=['SepalLength','SepalWidth','PetalLength','PetalWidth','Species']
species=['Setosa','Versicolor','Virginica']

In [42]:
#storing dataset in the keras dataset folder with the name mentioned in the get_file()

train_path=tf.keras.utils.get_file(
"iris_training.csv","https://storage.googleapis.com/download.tensorflow.org/data/iris_training.csv")

test_path=tf.keras.utils.get_file(
"iris_test.csv","https://storage.googleapis.com/download.tensorflow.org/data/iris_test.csv")

In [43]:
train_path

'C:\\Users\\hp\\.keras\\datasets\\iris_training.csv'

In [44]:
train=pd.read_csv(train_path,names=csv_column_names,header=0)
test=pd.read_csv(test_path,names=csv_column_names,header=0)



In [45]:
train.head() #print first 5 rows of dataframe

,SepalLength,SepalWidth,PetalLength,PetalWidth,Species
0,6.4,2.8,5.6,2.2,2
1,5.0,2.3,3.3,1.0,1
2,4.9,2.5,4.5,1.7,2
3,4.9,3.1,1.5,0.1,0
4,5.7,3.8,1.7,0.3,0


In [46]:
train_y=train.pop('Species')  #train labels
test_y=test.pop('Species')    #test labels

train.head()

,SepalLength,SepalWidth,PetalLength,PetalWidth
0,6.4,2.8,5.6,2.2
1,5.0,2.3,3.3,1.0
2,4.9,2.5,4.5,1.7
3,4.9,3.1,1.5,0.1
4,5.7,3.8,1.7,0.3


In [47]:
train.head()

,SepalLength,SepalWidth,PetalLength,PetalWidth
0,6.4,2.8,5.6,2.2
1,5.0,2.3,3.3,1.0
2,4.9,2.5,4.5,1.7
3,4.9,3.1,1.5,0.1
4,5.7,3.8,1.7,0.3


In [48]:
def input_fn(features,labels,training=True,batch_size=256):
    
    #convert the inputs to a Datset
    dataset= tf.data.Dataset.from_tensor_slices((dict(features),labels))
    
    #shuffle amd repeat if you are in training mode
    if training:
        dataset=dataset.shuffle(1000).repeat()
    #repeat will repeat all the elements of datset again and again indefinitely
    #repeat() does step into an infinite loop. 
    #If you wonder how training could have been finished, the documentation indicates that steps argument has been provided in
    #classifier.train(input_fn=lambda: input_fn(train, train_y, training=True), steps=5000) so even though the loop is infinite, the optimizer know how many steps to perform.
        
    ds= dataset.batch(batch_size)  #specify the size of batch to be made
    
    return ds


In [49]:
#Feature columns descripbe how to use the input

my_feature_columns=[]

#train.keys() gives list of all the columns of the train dataframe

for key in train.keys():
    my_feature_columns.append(tf.feature_column.numeric_column(key))
    
print(my_feature_columns)
    

[NumericColumn(key='SepalLength', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='SepalWidth', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='PetalLength', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='PetalWidth', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None)]


In [50]:
#build a DNN with 2 hidden layers with 30 and 10 hidden nodes

classifier=tf.estimator.DNNClassifier(
    feature_columns=my_feature_columns,
    #two hidden layers of 30 and 10 nides respectively
    hidden_units=[30,10],
    #the model must choose between 3 classes
    n_classes=3
)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'C:\\Users\\hp\\AppData\\Local\\Temp\\tmp2yxt46u1', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [51]:
classifier.train(input_fn=lambda:input_fn(train,train_y,training=True),steps=5000)
#lambda is a anonymous function that can be used to write a one line function
#train model requires a input_function object
#lambda returns a input function object
#look at the datset 5000 times
#during one step, assuming that my batch size is 20, it means that 20 pictures will be processed and will be used to update the gradients of my model once.

INFO:tensorflow:Calling model_fn.

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Calling checkpoint listeners before saving checkpoint 0...
INFO:tensorflow:Saving checkpoints for 0 into C:\Users\hp\AppData\Local\Temp\tmp2yxt46u1\model.ckpt.
INFO:tensorflow:Calling checkpoint listeners after saving checkpoint 0...
INFO:tensorflow:loss 

In [52]:
eval_result=classifier.evaluate(input_fn=lambda:input_fn(test,test_y,training=False))


INFO:tensorflow:Calling model_fn.

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2021-06-16T18:58:13Z
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\hp\AppData\Local\Temp\tmp2yxt46u1\model.ckpt-5000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Inference Time : 0.24870s
INFO:tensorflow:Finished evaluation at 2021-06-16-18:58:13
INFO:tensorflow:Saving dict for global step 5000: acc

In [53]:
eval_result

{'accuracy': 0.9,
 'average_loss': 0.5126056,
 'loss': 0.5126056,
 'global_step': 5000}

In [54]:
def input_fn(features,batch_size=256):
    #convert the inputs to a Dataset withut labels
    return tf.data.Dataset.from_tensor_slices(dict(features)).batch(batch_size)

In [63]:
features=['SepalLength','SepalWidth',"PetalLength","PetalWidth"]

predict={}

print("please type numeric values as prompted")
for feature in features:
    val=input(feature + ": ")
    predict[feature]=[float(val)]
    
prediction=classifier.predict(input_fn=lambda:input_fn(predict))
for pred_dict in prediction:
    class_id=pred_dict["class_ids"][0]
    probability=pred_dict["probabilities"][class_id]
    
 
    print(pred_dict)
    print(species[class_id],100*probability)
                
            

please type numeric values as prompted
SepalLength: 6.9
SepalWidth: 3.1
PetalLength: 5.4
PetalWidth: 2.1
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\hp\AppData\Local\Temp\tmp2yxt46u1\model.ckpt-5000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
{'logits': array([-0.56559944,  1.3770562 ,  2.1247473 ], dtype=float32), 'probabilities': array([0.04402563, 0.3071781 , 0.64879626], dtype=float32), 'class_ids': array([2], dtype=int64), 'classes': array([b'2'], dtype=object), 'all_class_ids': array([0, 1, 2]), 'all_classes': array([b'0', b'1', b'2'], dtype=object)}
Virginica 64.87962603569031
